In [21]:
def frequentPatternMining(tokens, tokensEncode, threshold, maxlen):
    dict_token = {}
    all_token={}
    
    tokensNumber = len(tokens)
    for i in range(tokensNumber):
        token = tokens[i]
        if token == '$':
            continue
        if token in dict_token:
            dict_token[token][0].append(i)
            dict_token[token][1].append(1)
        else:
            dict_token[token] = [[i],[1]]
    print ("# of distinct tokens = ", len(dict_token))

    token_pattern={}

    frequentPatterns = []
    patternLength = 1
    while (len(dict_token) > 0):
        if patternLength > maxlen:
            break
        print ("working on length = ", patternLength)
        patternLength += 1
        newDict = {}
        for pattern, positions_tf in dict_token.items():
            positions=positions_tf[0]
            valid=positions_tf[1]
            occurrence = len(positions)
            if occurrence >= threshold:
                
                if patternLength>2 and sum(valid)>threshold:
                    for j, p in enumerate(positions):
                        if valid[j]>0:
                            if p in token_pattern:
                                token_pattern[p].append(pattern)
                            else:
                                token_pattern[p]=[pattern]
                                
                    
                    frequentPatterns.append([pattern,occurrence])
                    all_token[pattern]=positions
                
                #patternOutput.write(pattern + "," + str(occurrence) + "\n")
                for i in positions:
                    if i + 1 < tokensNumber:
                        if tokens[i + 1] == '$':
                            continue
                        newPattern = pattern + " " + tokens[i + 1]
                        invalid=False
                        pattern_tree=tokensEncode[i-patternLength+2: i+2]
                        pattern_root=min(pattern_tree, key=len)
                        for t in pattern_tree:
                            if t[0:-1] in pattern_tree or t==pattern_root:
                                continue
                            else:
                                invalid=True
                                
                        if invalid:
                            new_valid=0
                        else:
                            new_valid=1
                         
                        if newPattern in newDict:
                            newDict[newPattern][0].append(i + 1)
                            newDict[newPattern][1].append(new_valid) 
                        else:
                            newDict[newPattern] = [[i + 1], [new_valid]]


                    
                    
        dict_token.clear()
        dict_token = newDict

    return sorted(frequentPatterns, key=lambda x:(x[0],x[1])), all_token, token_pattern

In [14]:
path = 'bio'
f = 'subSentence.'
f = 'new_sub1.'
ex_id='/train2.'
subOutput = open(path +ex_id+ f + 'txt', 'r')
tokens=[]
main_words=[]
for line in subOutput:
    sn = line.rstrip('\n').split(' \t ')
    snID+=1
    if len(sn)<2:
        tokens.append('$')
        main_words.append('$')
        continue
    else:
        main_words.append(sn[0])
        arr=sn[1].split(' ')
        for i in range(len(arr)):
           
            arr[i]=arr[i].split('_')
            if 'CHEMICAL' in arr[i][0]:
                arr[i][0]='CHEMICAL'
                tokens.append('_'.join(arr[i]))
            elif 'GENE' in arr[i][0]:
                arr[i][0]='GENE'
                tokens.append('_'.join(arr[i]))
            elif 'SPECIES' in arr[i][0] :
                arr[i][0]='SPECIES'
                tokens.append('_'.join(arr[i]))
            elif 'DISEASE' in arr[i][0]:          
                arr[i][0]='DISEASE'
                tokens.append('_'.join(arr[i]))
            
            else:
                tokens.append('_'.join(arr[i]))
    tokens.append('$')
#frequentPatterns = frequentPatternMining(tokens, patternOutputFilename, threshold)



In [15]:
l = 'subEncode.'
encodeOutput = open(path +ex_id+ l + 'txt', 'r')
snID=0
tokensEncode=[]
for line in encodeOutput:
    sn = line.rstrip(' .\n').split(' \t ')
    snID+=1
    if len(sn)<2:
        if sn:
            tokensEncode.append(sn[0])
        else:
            tokensEncode.append('$')
        continue
    else:
        arr=sn[1].split(' ')
        for i in range(len(arr)):
            tokensEncode.append(arr[i])
    tokensEncode.append('$')

In [11]:
if len(tokensEncode)!=len(tokens):
    for i in range(len(tokensEncode)):
        if (tokensEncode[i]=='$' and tokens[i]!='$') or (tokensEncode[i]!='$' and tokens[i]=='$'):
            print(tokens[i-10:i+10], tokens[i])
            print(tokensEncode[i-10:i+10], tokensEncode[i])
            print(i)
            break

In [16]:
f_out=path +ex_id+'metapattern.txt'
f_out=path +ex_id+'new_metapattern.txt'
frequentPatterns, pattern_location_dict, exact_match_dict=frequentPatternMining(tokens,tokensEncode,10,15)
write=True
if write:
    with open(f_out, 'w') as patternOutput:
        for p in frequentPatterns:
            line=p[0]+'\t'+str(p[1])
            patternOutput.write(line + '\n')


# of distinct tokens =  110189
working on length =  1
working on length =  2
working on length =  3
working on length =  4
working on length =  5
working on length =  6
working on length =  7
working on length =  8
working on length =  9
working on length =  10


In [32]:
f_out=path +ex_id+'pattern_match.txt'
f_out=path +ex_id+'new_pattern_match.txt'
k=0
pattern=[]
with open(f_out, 'w') as patternOutput:
    for j in range(len(tokensEncode)):
        
        if tokensEncode[j]=='$' and main_words[k]=='$':
            k+=1
            patternOutput.write('\n')
        elif tokensEncode[j]=='$' and main_words[k]!='$':
            
            patterns='\t'.join(pattern)
            patternOutput.write(main_words[k] + '\t' + patterns+'\n')
            pattern=[]
            k+=1
            #patternOutput.write('\n')
        elif tokensEncode[j]!='$' and tokens[j]=='$':
            k+=1
            patternOutput.write(tokensEncode[j] + '\n')
        else:
            if j in exact_match_dict:
                pattern_t=sorted(exact_match_dict[j], key=len, reverse=True)[0]
                if len(pattern)==0 or not pattern[-1] in pattern_t:
                    pattern.append(pattern_t)
                else:
                    pattern[-1]=pattern_t

        
        

In [35]:
j=0
for i in exact_match_dict:
    print(i,exact_match_dict[i])
    j+=1
    if j>10:
        break

2 ['effect of']
36 ['effect of', 'the effect of']
1482 ['effect of', 'the effect of']
1532 ['effect of']
2020 ['effect of', 'the effect of']
2912 ['effect of', 'side effect of']
3029 ['effect of']
4088 ['effect of', 'the effect of']
4125 ['effect of', 'the effect of']
4258 ['effect of', 'the effect of']
4323 ['effect of', 'stimulatory effect of', 'the stimulatory effect of']


In [13]:
f = 'metapattern.'
f = 'new_metapattern.'
g = 'stopword.'
inputPattern = open(path +ex_id+ f + 'txt', 'r')
stopwordFile = open(path +'/' + g + 'txt', 'r')
ner_types=['DISEASE','GENE', 'CHEMICAL','SPECIES']
stopword=[]
for line in stopwordFile:
    sn = line.rstrip('\n')
    stopword.append(sn)
tokens=[]
main_words=[]

multiNER = []
singleNER = []
phrase = []
for line in inputPattern:
    sn = line.rstrip('\n').split('\t')
    main_pattern = sn[0]
    nerNo=0
    for ner in ner_types:
        if ner in main_pattern:
            nerNo+=1
            r=main_pattern.split(ner)
            nerNo+=len(r)-2
    if nerNo<2 and nerNo>0:
        #print(line)
        r=main_pattern.split(' ')
        wordNo=len(r)
        for word in r:
            if word in stopword:
                wordNo-=1
        if wordNo>1:
            singleNER.append([main_pattern,int(sn[1])])
    elif nerNo==0:
        r=main_pattern.split(' ')
        wordNo=len(r)
        for word in r:
            if word in stopword:
                wordNo-=1
        if wordNo>1:
            phrase.append([main_pattern,int(sn[1])])
    else:
        multiNER.append([main_pattern,int(sn[1])])

        
singleNER.sort(key=lambda x: x[1],reverse=True)
multiNER.sort(key=lambda x: x[1],reverse=True)
phrase.sort(key=lambda x: x[1],reverse=True)
singlePath=path +ex_id + 'single.pattern.' + 'txt'
multiPath=path + ex_id + 'multi.pattern.' + 'txt'
phrasePath=path + ex_id + 'phrase.pattern.' + 'txt'
singlePath=path +ex_id + 'new_single.pattern.' + 'txt'
multiPath=path + ex_id + 'new_multi.pattern.' + 'txt'
phrasePath=path + ex_id + 'new_phrase.pattern.' + 'txt'
with open(singlePath, 'w') as single:
    for p in singleNER:
        single.write(p[0]+'\t'+str(p[1])+'\n')
with open(multiPath, 'w') as multi:
    for p in multiNER:
        multi.write(p[0]+'\t'+str(p[1])+'\n')     
with open(phrasePath, 'w') as ph:
    for p in phrase:
        ph.write(p[0]+'\t'+str(p[1])+'\n')
        